In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment

In [2]:
audio = AudioSegment.from_file(fr"cal_numerico_trab_03_audio_01.m4a", format="m4a")

In [3]:
# Printando atributos
print(f"Channels: {audio.channels}")
print(f"Sample width: {audio.sample_width}")
print(f"Frame rate (sample rate): {audio.frame_rate}")
print(f"Frame width: {audio.frame_width}")
print(f"Length (ms): {len(audio)}")
print(f"Frame count: {audio.frame_count()}")
print(f"Intensity: {audio.dBFS}")

Channels: 1
Sample width: 2
Frame rate (sample rate): 48000
Frame width: 2
Length (ms): 3051
Frame count: 146432.0
Intensity: -34.76046225140812


In [ ]:
#Extraindo dados do audio

audio = audio[500:1500] #Selecionando apenas 1s do audio
freq = audio.frame_rate
dt = 1000/freq #Tamanho dos segmentos em ms
tempo = len(audio) #Tempo total, em ms

samples = audio.get_array_of_samples()

#Criando array com pontos (instante, intensidade)
pontos = np.zeros((len(samples),2))
pontos[:,1] = samples
for i in range(len(samples)):
    pontos[i,0] = i*dt

n = int(input("Escreva o valor de n: ")

#Calculando A[0,0], b[0] e x[0]
A = [len(pontos)]
b = [np.sum(pontos, axis = 0)[1]]

#Calculando A[i,i] e b[i]
doispi_t = 2 * np.pi / tempo
#for comeca no 1 pois o termo 0 ja foi calculado
for i in range(1, n+1):
    resultadocos,resultadosin = 0,0
    resultado_bcos,resultado_bsin = 0,0

    #Somente um for e necessario para calcular todos os valores!
    for instante in pontos:
        resultadosin += (np.sin(i * doispi_t * instante[0] ))**2
        resultadocos += (np.cos(i * doispi_t * instante[0] ))**2
        resultado_bsin += instante[1] * (np.sin(i * doispi_t * instante[0] ))
        resultado_bcos += instante[1] * (np.cos(i * doispi_t * instante[0] ))

    #Note que os senos ocupam n impares e os cossenos ocupam n pares
    A += [resultadosin]
    A += [resultadocos]
    b += [resultado_bsin]
    b += [resultado_bcos]

#Transformando listas em arrays para calcular x
Aaux = np.zeros((2*n+1, 2*n+1))
np.fill_diagonal(Aaux,A)
b = np.array(b)
x = np.linalg.solve(Aaux,b)

#Criando |a_n + b_n| que sera plotado
vetx = [x[0]] + [abs(x[i]+x[i+1]) for i in range(1, 2*n + 1, 2)]

#Escrevendo x num arquivo txt
file = open(fr"vetor_x.txt", "x")
for i in vetx:
    file.write(str(i) + "\n")
file.close()

#Plotando grafico |a_n + b_n| vs. n
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.title(r"$F(x) = \displaystyle\sum_{n=0}^\infty a_n \cos \left(\frac{2 \pi}{T} n x \right)$"
            r"$\displaystyle + b_n \sin \left(\frac{2 \pi}{T} n x \right)$, $T=1s$, n=" f"{n}",
            fontsize=12)

plt.xlabel("n", fontsize=12)
plt.ylabel(r"$|a_n + b_n|$", fontsize=12)
plt.plot(np.arange(0, n+1, 1), vetx, color = "teal")

plt.show()